GET SD Filings for all CIKs we have

In [1]:
import feedparser
import requests
import os
import pandas as pd
import json

from random import randint
from time import sleep
from tqdm import tqdm

In [2]:
cik_list = pd.read_csv('cik_list.csv')
cik_list.tail(2)

,cik
7898,1485922
7899,1690511


In [4]:
def wait_random_time():
    # Average of 0.5s wait
    sleep(randint(0, 100)/100)

def get_edgar_feed_url(cik):
    base_url = "https://www.sec.gov/cgi-bin/browse-edgar"
    params = {
        'action': 'getcompany',
        'CIK': cik,
        'type': 'SD',
        'owner': 'exclude',
        'start': 0,
        'count': 40,
        'output': 'atom',
        'datea': f'20140101',
    }
    req = requests.Request('GET', base_url, params=params)
    prepped = req.prepare()
    feed_url = prepped.url
    return feed_url

def get_feed(cik):
    url = get_edgar_feed_url(cik)
    feed = feedparser.parse(url)
    return feed

In [4]:
for cik in tqdm(cik_list.cik.values):
    out_file = f'data/edgar_results/{cik}.json'
    if os.path.exists(out_file):
        continue
    wait_random_time()
    feed = get_feed(cik)
    json.dump(feed, open(out_file, 'w'))

100%|██████████| 7900/7900 [2:03:21<00:00,  1.07it/s]   


GET pages for each SD filing

In [2]:
df = pd.read_csv('summary_data/sd-filing-info-all.csv', index_col=0)
df.head()

,link,size,accession_number,filing_date,year,cik,conformed_name
0,https://www.sec.gov/Archives/edgar/data/147473...,45 KB,0001437749-19-011154,2019-05-31,2019,1474735,GENERAC HOLDINGS INC.
1,https://www.sec.gov/Archives/edgar/data/147473...,39 KB,0001437749-18-010977,2018-05-31,2018,1474735,GENERAC HOLDINGS INC.
2,https://www.sec.gov/Archives/edgar/data/147473...,48 KB,0001437749-17-010176,2017-05-26,2017,1474735,GENERAC HOLDINGS INC.
3,https://www.sec.gov/Archives/edgar/data/147473...,47 KB,0001437749-16-033079,2016-05-31,2016,1474735,GENERAC HOLDINGS INC.
4,https://www.sec.gov/Archives/edgar/data/147473...,46 KB,0001437749-15-011220,2015-05-28,2015,1474735,GENERAC HOLDINGS INC.


In [5]:
for _, row in tqdm(df.iterrows(), total=len(df)):
    out_file = f'data/edgar_pages/{row.accession_number}.html'
    if os.path.exists(out_file):
        continue
    wait_random_time()
    r = requests.get(row.link)
    if r.status_code == 200:
        with open(out_file, 'w') as f:
            f.write(r.text)
    else:
        print(f'{row.accession_number} gave error {r.status_code}')

100%|██████████| 6730/6730 [1:35:35<00:00,  1.17it/s]  
